In [2]:
import urllib.request
import json
 
# Your primary key (replace with secondary if you want)
api_key = '72fNK2ogONHXje1XShvxvQ3mZUKZN8kErOTKnZBx1xaEd9apKnoZJQQJ99BFAAAAAAAAAAAAINFRAZML3cS9'
 
if not api_key:
    raise Exception("A key should be provided to invoke the endpoint")
 
# Example input data - adjust this according to your model's expected input format
data = {
        "text": "This is a test sentence to evaluate the DeBERTa endpoint."
    }
 
# Encode the data to JSON bytes
body = str.encode(json.dumps(data))
 
# Your endpoint URL
url = 'http://194.171.191.227:30526/api/v1/endpoint/deberta-emotion-clf-endpoint/score'
 
# Set the request headers including authorization with your API key
headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
    'Authorization': 'Bearer ' + api_key
}
 
# Prepare the request
req = urllib.request.Request(url, body, headers)
 
try:
    # Send the request and read the response
    response = urllib.request.urlopen(req)
    result = response.read()
    print("Response from DeBERTa endpoint:")
    print(result.decode("utf-8"))
 
except urllib.error.HTTPError as error:
    # If there's an HTTP error, print status and details
    print("The request failed with status code:", error.code)
    print("Headers:", error.info())
    print("Error response:", error.read().decode("utf8", 'ignore'))

Response from DeBERTa endpoint:
"{\"predictions\": {\"emotion\": \"unknown\", \"sub_emotion\": \"unknown\", \"intensity\": \"unknown\"}, \"raw_predictions\": {\"emotion\": [[-2.3535149097442627, -2.7075352668762207, -2.1163432598114014, 2.768012762069702, 3.4213383197784424, -2.740438461303711, 1.3640868663787842]], \"sub_emotion\": [[-0.908567488193512, 0.7767903208732605, -4.667848587036133, -0.5953039526939392, -0.41957715153694153, -0.660086452960968, 1.8608988523483276, 4.128921985626221, -2.0373587608337402, -1.554884910583496, -1.1481451988220215, -3.3077480792999268, -3.654724597930908, -0.7455893754959106, -2.2528023719787598, -4.40749454498291, -5.066255569458008, -2.410703182220459, -4.219160079956055, -0.38470709323883057, 4.7725701332092285, 0.5356895327568054, -0.6146451234817505, 2.2942633628845215, 0.11964304745197296, -4.125598907470703, -2.3472158908843994, -0.8247337937355042]], \"intensity\": [[3.7233965396881104, 1.0488967895507812, -3.915595293045044]]}, \"confide